<a href="https://colab.research.google.com/github/vladimir-sashin/Recsys-course-homework/blob/master/RecSysLab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1Em-UZriSKRwCPahc5yLaaPPE3eESaa7S

Downloading...
From: https://drive.google.com/uc?id=1Em-UZriSKRwCPahc5yLaaPPE3eESaa7S
To: /content/data.csv
100% 2.47G/2.47G [00:26<00:00, 93.3MB/s]


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from IPython.display import display

In [2]:
df = pd.read_csv("data.csv")
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


# Подготовка данных

Произведем предобработку данных:

- Отсортируем датасет по дате/времени и разобъем данные на train и test сеты. Test - показы рекламы в последний день, в которых banner_id == banner_id0, train - все остальные.
- Оставим в них только те столбцы, которые нужны для обучения модели и получения предсказаний
- Cоздадим второе тестовое множество, в котором banner_id сделаем равным banner_id1
- Создадим отдельный датафрейм со столбцами "coeff_sum0", "g0", "coeff_sum1", "g1" для последнего дня наблюдений, который пригодится при подсчете pi_0 и pi_1

In [3]:
df["date_time"] = pd.to_datetime(df.date_time)
df = df.sort_values("date_time")
max_date = df["date_time"].iloc[-1].date()
df.drop(index=df.iloc[:1, :].index.tolist(), inplace=True)

df_test = df[(df['date_time'].dt.date == max_date) & (df["banner_id"] == df["banner_id0"])].copy()
df_for_pi = df_test[["coeff_sum0", "g0", "coeff_sum1", "g1"]].copy()

df_test_for_2nd_banner = df_test.copy()
df_test_for_2nd_banner["banner_id"] = df_test_for_2nd_banner["banner_id1"]
cols_to_drop = ["oaid_hash", "banner_id0", "banner_id1", "rate0", "rate1", "g0",
              "g1", "coeff_sum0", "coeff_sum1", "impressions", "campaign_clicks"]
df_test, df_test_for_2nd_banner = df_test.drop(cols_to_drop, axis=1), df_test_for_2nd_banner.drop(cols_to_drop, axis=1)


df = df[(df['date_time'].dt.date < max_date)]
df = df.drop(["oaid_hash", "banner_id0", "banner_id1", "rate0", "rate1", "g0",
              "g1", "coeff_sum0", "coeff_sum1", "impressions", "campaign_clicks"], axis=1)

In [4]:
df_for_pi

,coeff_sum0,g0,coeff_sum1,g1
14196412,-6.356145,0.011367,-6.355855,0.011368
8706638,-3.107591,0.063288,-3.091600,0.041912
13000378,-3.023266,0.042959,-3.060019,0.042071
9767447,-4.665202,0.051014,-4.988883,0.014532
9054327,-3.672826,0.157651,-3.676580,0.167411
...,...,...,...,...
11510884,-6.085725,0.009155,-2.452728,0.081177
10139863,-3.791884,0.038556,-3.749882,0.026928
5597133,-4.641030,0.023820,-2.737213,0.010748
8660907,-5.617197,0.027977,-5.616952,0.027988


# Feature engineering

1. Количество предшествующих показов той же кампании удалим, в отличие от первой работы
2. Из даты и времени показа вытащим две фичи - день недели и час
3. Сконструируем следующие интеракции, то есть создадим новые признаки показа из существующих путем их объединения:
- баннер, ОС, место на сайте - какой баннер, на какой платформе и в каком месте был показан
- баннер, страна, время - в какой стране и в какое время дня конкретный баннер был выведен при данном показе (таким образом учитываем различные таймзоны)
- день недели, баннер - какой баннер был показан в определенное время дня
- баннер, страна - какой баннер в какой стране показывается
- баннер, место на сайте - какой баннер в каком месте сайта показывается
- день недели - просто в какой день недели был произведен показ
4. Уберем лишние столбцы
5. Отделим предикторы от отклика
6. Изменим типы данных предикторов


In [5]:
def feature_engineering(df):

    df["dayoftheweek"] = df["date_time"].dt.dayofweek
    df['hour'] = df["date_time"].dt.hour

    df['banner_os_zone'] = (df['banner_id'].astype('str') + '_' + df['os_id'].astype('str') + '_' + df['zone_id'].astype('str')).astype('category')
    df['banner_country_time'] = (df['banner_id'].astype('str') + '_' + df['country_id'].astype('str') + '_' + df['hour'].astype('str')).astype('category')
    df['banner_weekday'] = (df['banner_id'].astype('str') + '_' + df['dayoftheweek'].astype('str')).astype('category')
    df['banner_country'] = (df['banner_id'].astype('str') + '_' + df['country_id'].astype('str')).astype('category')
    df['banner_zone'] = (df['banner_id'].astype('str') + '_' + df['zone_id'].astype('str')).astype('category')
    df["dayoftheweek"] = df["dayoftheweek"].astype("category")

    df = df.drop(["date_time", "zone_id", "banner_id", "os_id", "country_id", "hour"], axis=1)
    

    y = df["clicks"].values
    df = df.drop(["clicks"], axis=1)

    categorical_feature_mask = df.dtypes == "category"
    categorical_cols = df.columns[categorical_feature_mask].tolist()
    return df, y, categorical_cols


df, y, categorical_cols = feature_engineering(df)

In [6]:
df.shape, y.shape

((13692493, 6), (13692493,))

In [7]:
df.head()

,dayoftheweek,banner_os_zone,banner_country_time,banner_weekday,banner_country,banner_zone
5041415,6,29_3_41,29_0_0,29_6,29_0,29_41
1442602,6,188_2_1,188_15_0,188_6,188_15,188_1
7232498,6,52_2_17,52_5_0,52_6,52_5,52_17
14938691,6,73_4_47,73_13_0,73_6,73_13,73_47
11536774,6,266_0_48,266_1_0,266_6,266_1,266_48


Произведем one-hot encoding всех наших фичей. Будем фитить класс OneHotEncoder из sklearn, чтобы потом получить те же столбцы и на тестовом сете, при этом установим sparse=True, чтобы на выходе получить sparse матрицу в формате csr для более эффективной работы с данными

In [8]:
from sklearn.preprocessing import OneHotEncoder


ohe = OneHotEncoder(handle_unknown='ignore', sparse = True)
ohe.fit(df[categorical_cols])
df = ohe.transform(df[categorical_cols])
df

<13692493x563791 sparse matrix of type '<class 'numpy.float64'>'
	with 82154958 stored elements in Compressed Sparse Row format>

# Обучение модели

Обучать будем логистическую регрессию с оптимизатором liblinear и l2 регуляризацией. Чтобы подобрать гиперпараметр регуляризации, проведем кросс-валидацию с разбиением на 3 подмножества, используя TimeSeriesSplit, чтобы тестовые данные всегда были позже по времени, чем обучающие. После того, как подберем гиперпараметр, обучим модель на целой тренировочной выборке

In [11]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from joblib import dump


model = LogisticRegression(random_state=42, penalty="l2", solver="liblinear")
param_search = {'C' : [0.0001, 0.001, 0.01, 0.1]}

splits = TimeSeriesSplit(n_splits=3)
gsearch = GridSearchCV(estimator=model, cv=splits,
                        param_grid=param_search, verbose=2, scoring="neg_log_loss")
gsearch.fit(df, y)

print(gsearch.cv_results_)

best_model = gsearch.best_estimator_
print("Best score: {} \nBest params: {}".format(gsearch.best_score_, gsearch.best_params_))

best_model.fit(df, y)
dump(best_model, 'fit_model_new.joblib') 

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ...........................................C=0.0001; total time=  10.9s
[CV] END ...........................................C=0.0001; total time=  20.7s
[CV] END ...........................................C=0.0001; total time=  29.6s
[CV] END ............................................C=0.001; total time=  14.0s
[CV] END ............................................C=0.001; total time=  26.8s
[CV] END ............................................C=0.001; total time=  49.2s
[CV] END .............................................C=0.01; total time=  22.0s
[CV] END .............................................C=0.01; total time=  55.4s
[CV] END .............................................C=0.01; total time= 1.5min
[CV] END ..............................................C=0.1; total time=  56.1s
[CV] END ..............................................C=0.1; total time= 2.0min
[CV] END ........................................

['fit_model_new.joblib']

# Получение предсказаний, оценка Logloss, расчет pi_0, pi_1 и CIPS

Сначала получим предикты на последний день и посчитаем logloss

In [12]:
df = df_test
df, y, categorical_cols = feature_engineering(df)

df = ohe.transform(df[categorical_cols])
df

<1890562x563791 sparse matrix of type '<class 'numpy.float64'>'
	with 7024937 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.metrics import log_loss


predictions = best_model.predict_proba(df)
log_loss(y, predictions)

0.1355446808589051

Посчитаем coeff_sum0_new

In [14]:
from scipy.special import logit


coeff_sum0_new = logit(np.array(predictions)[:, 1])
coeff_sum0_new

array([-8.12395368, -2.91527584, -2.74824109, ..., -4.15173616,
       -6.8919276 , -2.36502429])

Посчитаем coeff_sum1_new

In [15]:
df, y, categorical_cols = feature_engineering(df_test_for_2nd_banner)

df = ohe.transform(df[categorical_cols])
df

<1890562x563791 sparse matrix of type '<class 'numpy.float64'>'
	with 5293078 stored elements in Compressed Sparse Row format>

In [16]:
predictions = best_model.predict_proba(df)
coeff_sum1_new = logit(np.array(predictions)[:, 1])
coeff_sum1_new

array([-8.12496008, -2.97585613, -2.64966454, ..., -3.21605559,
       -6.8909006 , -2.34892925])

Считаем pi_0 и pi_1

In [17]:
from scipy.stats import norm


def count_pi(mean_1, mean_2, std_1, std_2):
    mean = mean_1 - mean_2
    std = np.sqrt(std_1 ** 2 + std_2 ** 2)
    return 1 - norm.cdf((0 - mean) / std, loc=0, scale=1)


pi_0 = count_pi(df_for_pi["coeff_sum0"], df_for_pi["coeff_sum1"], df_for_pi["g0"], df_for_pi["g1"])
pi_1 = count_pi(coeff_sum0_new, coeff_sum1_new, df_for_pi["g0"], df_for_pi["g1"])

len(pi_0), len(pi_1)

(1890562, 1890562)

In [18]:
pi_0

array([0.49278185, 0.41657691, 0.72948166, ..., 0.        , 0.4975297 ,
       0.99980879])

In [19]:
pi_1

array([0.524958  , 0.78758862, 0.05056191, ..., 0.        , 0.48964777,
       0.31867045])

Считаем CIPS

In [20]:
def cips(pi_0, pi_1, rewards, lambd):
    with np.errstate(all='ignore'):
        return 1 / len(pi_0) * np.sum(np.minimum(np.nan_to_num(pi_0 / pi_1, lambd), lambd) * rewards)

In [21]:
cips(pi_0, pi_1, y, 10)

0.0797509546788766